In [1]:
import openpathsampling as paths
import openpathsampling.engines.features as feats
from openpathsampling.experimental.storage.snapshots import schema_from_entries

In [2]:
toy_schema = schema_from_entries([feats.coordinates, feats.velocities, feats.engine], lazies=[])
toy_schema

{'snapshot': [('coordinates', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('velocities', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('engine', 'uuid')]}

In [3]:
statics_schema = schema_from_entries([feats.statics], ['statics'])
statics_schema

{'statics': [('coordinates', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('box_vectors', 'ndarray.float32({n_spatial},{n_spatial})')],
 'snapshot': [('statics', 'lazy')]}

In [4]:
import openpathsampling.engines.toy as toys

In [5]:
descriptor = frozenset([('n_atoms', 1000), ('n_spatial', 3), ('class', 'fuqyu')])

In [6]:
# this will probably be made into a propery of the snapshot
# descriptor is self.engine.descriptor
# schema is self._features_schema
def replace_schema_dimensions(schema, descriptor):
    descriptor_dict = {desc[0]: desc[1] for desc in descriptor
                       if desc[0] != 'class'}
    for (table, entries) in schema.items():
        schema[table] = [
            (attr, type_name.format(**descriptor_dict))
            for (attr, type_name) in entries
        ]
    return schema
        

In [7]:
replace_schema_dimensions(toy_schema, descriptor)

{'snapshot': [('coordinates', 'ndarray.float32(1000,3)'),
  ('velocities', 'ndarray.float32(1000,3)'),
  ('engine', 'uuid')]}

In [8]:
replace_schema_dimensions(statics_schema, descriptor)

{'statics': [('coordinates', 'ndarray.float32(1000,3)'),
  ('box_vectors', 'ndarray.float32(3,3)')],
 'snapshot': [('statics', 'lazy')]}

### Saving a snapshot when we haven't registered it yet

Note: this is now an EXPECTED failure... you must register data objects.

In [9]:
from openpathsampling.experimental.simstore import GeneralStorage
from openpathsampling.experimental.simstore import ClassInfoContainer, ClassInfo
from openpathsampling.experimental.simstore import universal_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
import openpathsampling.experimental.simstore.serialization_helpers as serialization
from openpathsampling.experimental.simstore.custom_json import JSONSerializerDeserializer
from openpathsampling.experimental.simstore.custom_json import numpy_codec, bytes_codec, uuid_object_codec
from openpathsampling.experimental.simstore import tools

import openpathsampling.engines.toy as toys

import numpy as np

In [10]:
codecs = [numpy_codec, bytes_codec, uuid_object_codec]
json_ser_deser = JSONSerializerDeserializer(codecs)

#json_serializer, json_deserializer = default_serializer_deserializer(
#    [numpy_codec, bytes_codec, uuid_object_codec]
#)
#json_serializer = serialization.to_json_obj
#serialize_sim = SimulationObjectSerializer(json_serializer)
serialize_sim = json_ser_deser.simobj_serializer

deserialize_sim = serialization.from_json_obj
serialize_data = serialization.to_dict_with_uuids
deserialize_data = lambda cls, data: cls.from_dict(serialization.from_dict_with)

In [11]:
from openpathsampling.netcdfplus import StorableObject
default_class_info = ClassInfo(table='simulation_objects',
                               cls=StorableObject,
                               serializer=serialize_sim,
                               deserializer=deserialize_sim)

In [12]:
backend = SQLStorageBackend(":memory:", mode='w')
storage = GeneralStorage(backend=backend,
                         schema={},
                         class_info=ClassInfoContainer(default_info=default_class_info))

In [13]:
schema = {
    'simulation_objects': [('json', 'json'), ('class_idx', 'int')]
}
storage.register_schema(schema, class_info_list=[])

In [14]:
pes = (
    toys.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

topology=toys.Topology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)
integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'n_steps_per_frame' : 1
}

engine = toys.Engine(
    options=options,
    topology=topology
)

In [15]:
template = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    engine=engine
)

In [16]:
storage.save(engine)

In [17]:
storage.save(template)

In [18]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'simulation_objects'])

In [19]:
sim_objs = backend.metadata.tables['simulation_objects']
with backend.engine.connect() as conn:
    res = [r for r in conn.execute(sim_objs.select())]

In [20]:
res

[(1, '318218588850860764244677641445350309910', '{"options": {"n_frames_max": 5000, "on_max_length": "fail", "on_nan": "fail", "retries_when_nan": 2, "retries_when_error": 0, "retries_when_max_lengt ... (120 characters truncated) ... ame": 1}, "topology": "UUID(318218588850860764244677641445350309904)", "__class__": "ToyEngine", "__module__": "openpathsampling.engines.toy.engine"}', None),
 (2, '318218588850860764244677641445350309904', '{"masses": [1.0, 1.0], "n_atoms": 1, "pes": "UUID(318218588850860764244677641445350309902)", "n_spatial": 2, "__class__": "ToyTopology", "__module__": "openpathsampling.engines.toy.topology"}', None),
 (3, '318218588850860764244677641445350309906', '{"gamma": 2.5, "temperature": 0.1, "dt": 0.02, "__class__": "LangevinBAOABIntegrator", "__module__": "openpathsampling.engines.toy.integrators"}', None),
 (4, '318218588850860764244677641445350309902', '{"pes1": "UUID(318218588850860764244677641445350309898)", "pes2": "UUID(318218588850860764244677641445350

with a general storage, this saves it in `simulation_objects`, since it is a `StorableObject` (need to move all simulation objects to `StorableNamedObject`!)

### With an OPSStorage

In [21]:
from openpathsampling.experimental.storage.ops_storage import Storage, OPSClassInfoContainer

In [22]:
backend = SQLStorageBackend(":memory:", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=schema,
    class_info=OPSClassInfoContainer(default_info=default_class_info)
)

In [23]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'simulation_objects'])

In [24]:
storage.save(template)

In [25]:
storage.class_info[template]

ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('318218588850860764244677641445350309910', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x117b874e0>)

In [26]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'simulation_objects', 'snapshot0'])

In [27]:
lookup = storage.class_info.lookup_key(template)
lookup

('318218588850860764244677641445350309910',
 openpathsampling.engines.toy.snapshot.ToySnapshot)

In [28]:
storage.class_info['snapshot0'].lookup_result

('318218588850860764244677641445350309910',
 openpathsampling.engines.toy.snapshot.ToySnapshot)

In [29]:
sim_objs = backend.metadata.tables['simulation_objects']
with backend.engine.connect() as conn:
    res = [r for r in conn.execute(sim_objs.select())]
res

[(1, '318218588850860764244677641445350309910', '{"options": {"n_frames_max": 5000, "on_max_length": "fail", "on_nan": "fail", "retries_when_nan": 2, "retries_when_error": 0, "retries_when_max_lengt ... (120 characters truncated) ... ame": 1}, "topology": "UUID(318218588850860764244677641445350309904)", "__class__": "ToyEngine", "__module__": "openpathsampling.engines.toy.engine"}', None),
 (2, '318218588850860764244677641445350309904', '{"masses": [1.0, 1.0], "n_atoms": 1, "pes": "UUID(318218588850860764244677641445350309902)", "n_spatial": 2, "__class__": "ToyTopology", "__module__": "openpathsampling.engines.toy.topology"}', None),
 (3, '318218588850860764244677641445350309906', '{"gamma": 2.5, "temperature": 0.1, "dt": 0.02, "__class__": "LangevinBAOABIntegrator", "__module__": "openpathsampling.engines.toy.integrators"}', None),
 (4, '318218588850860764244677641445350309902', '{"pes1": "UUID(318218588850860764244677641445350309898)", "pes2": "UUID(318218588850860764244677641445350

In [30]:
sim_objs = backend.metadata.tables['snapshot0']
with backend.engine.connect() as conn:
    res = [r for r in conn.execute(sim_objs.select())]
res

[(1, '318218588850860764244677641445350309912', b'\x00\x00\x00\x00\x00\x00\x00\x00', b'\x00\x00\x00\xbf\x00\x00\x00\xbf', '318218588850860764244677641445350309910')]

### Using the new `class_info`-based serialization

In [31]:
info = storage.class_info['snapshot0']
info.serializer(template)

{'velocities': b'\x00\x00\x00\x00\x00\x00\x00\x00',
 'coordinates': b'\x00\x00\x00\xbf\x00\x00\x00\xbf',
 'engine': '318218588850860764244677641445350309910',
 'uuid': '318218588850860764244677641445350309912'}

In [32]:
info.serializer.entries

[('velocities', 'ndarray.float32(1,2)'),
 ('coordinates', 'ndarray.float32(1,2)'),
 ('engine', 'uuid')]

In [33]:
info.serializer.__dict__

{'schema': {'snapshot0': [('velocities', 'ndarray.float32(1,2)'),
   ('coordinates', 'ndarray.float32(1,2)'),
   ('engine', 'uuid')]},
 'table': 'snapshot0',
 'entries': [('velocities', 'ndarray.float32(1,2)'),
  ('coordinates', 'ndarray.float32(1,2)'),
  ('engine', 'uuid')],
 'cls': openpathsampling.engines.toy.snapshot.ToySnapshot,
 'attribute_handlers': {'velocities': <function openpathsampling.experimental.storage.serialization.ToDictSerializer.make_numpy_handler.<locals>.<lambda>(arr)>,
  'coordinates': <function openpathsampling.experimental.storage.serialization.ToDictSerializer.make_numpy_handler.<locals>.<lambda>(arr)>,
  'engine': <function openpathsampling.experimental.storage.serialization_helpers.get_uuid(obj)>}}

In [34]:
info.find_uuids(template, {})

({'318218588850860764244677641445350309912': <openpathsampling.engines.toy.snapshot.ToySnapshot at 0x117b1c0f0>},
 [<openpathsampling.engines.toy.engine.ToyEngine at 0x117b1cf28>])

In [35]:
engine_uuid = serialization.get_uuid(template.engine)

In [36]:
known_uuids = {engine_uuid: template.engine}

In [37]:
info.find_uuids(template, known_uuids)

({'318218588850860764244677641445350309912': <openpathsampling.engines.toy.snapshot.ToySnapshot at 0x117b1c0f0>},
 [<openpathsampling.engines.toy.engine.ToyEngine at 0x117b1cf28>])